In [1]:
import importlib
import utils.data_loader
import utils.tokenize
import models.embedding
import models.nn

#For reloading
importlib.reload(utils.data_loader)
importlib.reload(utils.tokenize)
importlib.reload(models.embedding)
importlib.reload(models.nn)

Initialized from jupyter or other except terminal or command prompt script!


<module 'models.nn' from '/Users/bimalmgr/Downloads/deep_learning/transformers/trnsfmr_from_scratch/models/nn.py'>

In [2]:
from utils.data_loader import DataLoader
from utils.tokenize import Tokenizer
from data import en,es
from models.embedding import Embeddings
from data import en,es
import torch
import torch.nn as nn

In [3]:
def reload_modules():
    %load_ext autoreload
    %autoreload 2

In [4]:
reload_modules()
print("This is the implementation of transformer from scratch in PyTorch.........!!")
tokenizer = Tokenizer()
data_loader = DataLoader(
    tokenize_source=tokenizer.tokenize_source,
    tokenize_target=tokenizer.tokenize_target
)
# train_set,valid_set,test_set = data_loader.make_dataset(es,en)
# vocab_src,vocab_trg = data_loader.build_vocab(train_set,valid_set,test_set)
# for word,index in vocab_trg.get_stoi().items():
#     print(word,index)
# print(f"Tokenized source example: {tokenizer.tokenize_source('Mi nombre es Bimal !! @ 1 23 * ;.')}")
# print(f"Tokenized target example: {tokenizer.tokenize_target('My name is Bimal !! @ 1 23 * ;.')}")
# print(f"Dataset examples: {data_loader.make_dataset(['Mi nombre es Bimal !! @ 1 23 * ;.'],['My name is Bimal !! @ 1 23 * ;.'])}")
s_tr,s_v,s_te = data_loader.make_dataset(['Mi nombre ','es Bimal !!','@ 1 23 * ;.'],['My name','is Bimal !!','@ 1 23 * ;.'])
print("train",s_tr,"val",s_v,"test",s_te)
s_v,s_t = data_loader.build_vocab(s_tr,s_v,s_te)
print(s_v.get_stoi())
print(s_t.get_stoi())
# print("Source vobularies:",vocab_src)
embed = nn.Embedding(len(s_v),5)
print(embed(torch.tensor([s_v.get_stoi()['Bimal'],s_v.get_stoi()['nombre']])))
# print(embed(torch.tensor(vocab_src.get_stoi()['víctimas'])))


This is the implementation of transformer from scratch in PyTorch.........!!
Initializing the data loader........
train [{'src': ['<sos>', 'Mi', 'nombre', '<eos>'], 'trg': ['<sos>', 'My', 'name', '<eos>']}, {'src': ['<sos>', 'es', 'Bimal', '!', '!', '<eos>'], 'trg': ['<sos>', 'is', 'Bimal', '!', '!', '<eos>']}] val [] test [{'src': ['<sos>', '@', '1', '23', '*', ';', '.', '<eos>'], 'trg': ['<sos>', '@', '1', '23', '*', ';', '.', '<eos>']}]
Building Spanish Vocabulary.....
Done Building Spanish Vocabulary!
Buidling English Vocabulary.........
Done building English Vocabulary!
{'Bimal': 11, '23': 8, '1': 7, '.': 6, 'es': 13, '*': 5, '!': 4, ';': 9, '<sos>': 2, 'nombre': 14, 'Mi': 12, '@': 10, '<eos>': 3, '<pad>': 1, '<unk>': 0}
{'is': 13, 'Bimal': 11, '23': 8, '1': 7, 'name': 14, '.': 6, '*': 5, '!': 4, ';': 9, '<sos>': 2, '@': 10, '<eos>': 3, 'My': 12, '<pad>': 1, '<unk>': 0}
tensor([[ 1.6507, -0.7271,  0.1678, -2.0592, -0.4729],
        [-0.3511, -0.5692, -0.3795,  0.8179,  0.8439]],
 

In [5]:
m =  nn.Linear(5,5)
input = torch.randn(2,5)
print(f"Input: {input};\n Weights: {m.weight} \n Bias:{m.bias} \n After transformation: {m(input)}\n")

Input: tensor([[-0.6032, -0.1903, -0.0376,  2.1739,  0.7048],
        [ 1.1337, -1.0121, -0.0053,  0.7635, -0.6674]]);
 Weights: Parameter containing:
tensor([[-0.4285, -0.1749,  0.3861,  0.1003, -0.0440],
        [-0.3463,  0.3087, -0.0713, -0.2879,  0.1215],
        [-0.3353, -0.1929,  0.2763, -0.2470,  0.3143],
        [-0.3104,  0.0472, -0.3150, -0.1126, -0.2413],
        [ 0.0214,  0.4337, -0.2485, -0.4455, -0.0150]], requires_grad=True) 
 Bias:Parameter containing:
tensor([ 0.3792,  0.0900, -0.0541,  0.3431, -0.0840], requires_grad=True) 
 After transformation: tensor([[ 0.8436, -0.2974, -0.1409,  0.1184, -1.1492],
        [ 0.1745, -0.9155, -0.6388,  0.0202, -0.8275]],
       grad_fn=<AddmmBackward0>)



In [6]:
print(f"xA^T + b") # This transformation applies after the nn.Linear is applied.
x = torch.tensor([[-0.3074,  1.5631,  0.5804, -1.0671, -0.6132],
        [-0.1582, -0.2556, -0.1892,  0.0084,  0.7520]])
w = torch.tensor([[-0.1351,  0.0465,  0.2974,  0.0119, -0.1067],
        [-0.2380,  0.2958,  0.1583,  0.0208, -0.1320],
        [-0.3229, -0.1220, -0.2476,  0.1831,  0.1225],
        [ 0.0242,  0.0074,  0.0925, -0.2421, -0.2087],
        [ 0.2182, -0.0354,  0.2494, -0.3794, -0.4260]])
b = torch.tensor([-0.3079,  0.3990,  0.3536, -0.1749,  0.3743])
a = x @ w.t()
o = a + b
print(o)

xA^T + b
tensor([[ 0.0317,  1.0852, -0.1520,  0.2692,  1.0627],
        [-0.4348,  0.2320,  0.5764, -0.3571, -0.0219]])


In [ ]:
from models.scaled_dot_product_attention import ScaledDotProductAttention

embedded_bimal = embed(torch.tensor([s_v.get_stoi()['Bimal'],s_v.get_stoi()['My']]))
scaled_attention = ScaledDotProductAttention(5)
print(scaled_attention.forward(embedded_bimal,embedded_bimal,embedded_bimal))

In [8]:
print(f"This is the full implementation of transformers from scratch in pytorch")

This is the full implementation of transformers from scratch in pytorch
